Домашнее задание 2.
## Комсомольская правда

In [1]:
import lxml.html
import urllib.request as ur
import requests
import time
import re
import os
from bs4 import BeautifulSoup
import random

### Задание

скачать КП - тексты с июля 2015 по декабрь 2016

сами тексты отдельными файлами + метаданные в таблице csv табуляцией

Метаданные: название статьи, автор, дата, количество слов, путь к файлу

сами тексты должны быть распределены по папкам по году и по месяцу в году

скачивание с помощью lxml

Корпус должен быть очищен и продублирован - plain text и с размееткой  mystem через os.system

## Сбор текстов

In [2]:
#задаем хэдеры - они понадобятся еще много раз
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/600.3.18 (KHTML, like Gecko) Version/8.0.3 Safari/600.3.18'
headers = { 'User-Agent' : user_agent }

In [3]:
#startlink - первая ссылка, с которой мы начнем наш обход ресурса
# мне повезло, у КП все тексты имеют в URL айди, соответствующий номеру статьи на ресурсе

startlink = "http://www.kp.ru/online/news/2591074/"

In [7]:
# создаем папки с годами - 2015 и 2016, и в каждой по 12 папок с месяцами

for year in range(2015,2017):
    for month in range(1,13):
        if month < 10:
            month = "0" + str(month)
        path = "/home/mi_air/Downloads/Komsomol_Truth/plain_text/"+str(year)+"/"+str(month)
        os.makedirs(path, mode=0o777, exist_ok=False)
        path_mystem = "/home/mi_air/Downloads/Komsomol_Truth/mystem/"+str(year)+"/"+str(month)
        os.makedirs(path_mystem, mode=0o777, exist_ok=False)

In [5]:
# файлы будем называть айдишниками текстов, так как они уникальные
# обход начнем с указанного номера 
num = int(startlink.split("/")[5])

In [3]:
#напишем функцию, которая будет проверять, находится ли по указанной ссылке текст, в нужном интервале дат:
# с июля 2015 по декабрь 2016 (июль 2015 включаем, декабрь 2016 не включаем)
# заодно она будет возвращать нам год с месяцем, чтобы удобно было сохранять текст в нужную папку
def TextDate(link):
    bl = False
    try:
        r = requests.get(link)
        date=re.split("T", re.split("'pubdate':'", r.text)[1])[0]
        year = int(re.split("-", date)[0])
        month = int(re.split("-", date)[1])
        if year==2016:
            if month<12:
                bl=True
        elif year==2015:
            if month>5:
                bl=True
        return [bl,year,month]
    except:
        return [bl]

In [4]:
#проверка этой функции
print(TextDate("http://www.kp.ru/online/news/2644468/"))
print(TextDate("http://www.kp.ru/online/news/1844468/"))
print(TextDate("http://www.kp.ru/online/news/2144468/"))
print(TextDate("http://www.kp.ru/online/news/2644445/")) #удаленная статья

[False, 2017, 2]
[False, 2014, 9]
[True, 2015, 8]
[False]


## Извлечение текстов

In [5]:
#напишем функцию, извлекающую текст и метаданные по ссылке
def getArticleTextKP(adr):
    resp = ur.urlopen(adr)
    page = resp.read().decode("utf-8")
    resp.close()
    tree = lxml.html.fromstring(page)
    #со структурой повезло не так сильно: вся основная метаинформация доступна, но лежит текстовым
    # словарем внутри dataLayer в теге <script>
    
    author = re.split("',", re.split("'authors\': \'", tree.xpath('.//script[@type="text/javascript"][2]/text()')[0])[1])[0]
    date  = re.split("T", re.split("'pubdate':'", tree.xpath('.//script[@type="text/javascript"][2]/text()')[0])[1])[0]
    #нужен формат дд.мм.гггг
    date2 = date.split("-")
    date = ".".join(date2[::-1])
    source = "Комсомольская Правда"
    title = tree.findtext(".//title")
    url = adr
    
    text = re.split("</div></article>",re.split('itemprop="articleBody" id="hypercontext">', page)[1])[0] 
    #текст доставала так, потому что lxml выкусывал ссылки на именованные сущности внутри текста. зато почистила потом 
    # режимом lxml в BeautifulSoup:
    beaux_text=BeautifulSoup(text, "lxml")
    text=beaux_text.get_text() 
    text = re.sub("\n", "", text)
    wordcount = str(len(text.split(" ")))
    
    #time.sleep(random.uniform(0, 2)) #чтобы нас не забанили

    return [author, date, source, title, url, wordcount, text] 

In [6]:
metatable = open(r'/home/mi_air/Downloads/Komsomol_Truth/metatable.csv', "w", encoding="utf-8")
firstline = 'path;author;date;source;title;url;wordcount' 
metatable.write(firstline + '\n')

44

In [10]:
#при проверке функции TextDate мы выяснили, что в районе 1,8 млн статей нужные нам статьи уже заканчиваются, 
#поэтому не будет заставлять программу перебирать номера статей так долго и ограничим их этим числом

while num>1866621:
    link = "http://www.kp.ru/online/news/"+str(num)+"/"
    
    if TextDate(link)[0]==True:
        
        month,year = TextDate(link)[2],TextDate(link)[1]
        if month<10:
            month = "0"+str(month)
        
        try:
            #author;date;source;title;url;wordcount
            params = getArticleTextKP(link)
            metaline = ";".join(params[:6])
            text = params[6]
            filename = '/home/mi_air/Downloads/Komsomol_Truth/plain_text/' + str(year) + "/" + str(month)+"/" + str(num) + ".txt"
            metaline = filename + ";" +  metaline 
            metatable.write(metaline + "\n")
            textfile = open(filename, "w", encoding="utf-8")
            for line in text:
                textfile.write(line)
            #print(filename)
        except:
            pass
    num-=17 #в нескольких запусках шаг также был 100, 3 и 1

## Морфологическая разметка

In [2]:
# теперь обойдем все папки с plain text и вызовем mystem для файлов в них
for top, dirs, files in os.walk('/home/mi_air/Downloads/Komsomol_Truth/plain_text/'):
    for file in files:
        filename = os.path.join(top, file)
        outfilename = re.sub("plain_text", "mystem", filename)
        os.system('/home/mi_air/mystem -ni -d ' +filename + " " + outfilename)
        # print(outfilename)
        